# Simulation 4 with dynamic aggregation

In [ ]:
"""
Simulating the WTSN setting

Authors: Milind Kumar Vaddiraju, ChatGPT, Copilot
"""

# Necessary imports
import copy
from datetime import datetime
import json
# %matplotlib inlinez
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import sys
import time

sys.path.insert(0, os.path.abspath('..'))

from network_classes import *
from utils import *
from schedules import *

%load_ext line_profiler

In [ ]:
parameters_filename = "../wireless_parameters/wireless_parameters_DL_MU_964B_80MHz.json"
with open(parameters_filename, 'r') as f:
    parameters = json.load(f)
print(parameters["setting 6"]["delivery_latency"][9])

delivery_latency_OFDMA = {}
# TODO: Check MCS numbers
delivery_latency_OFDMA["MCS 2"] = [1172.8, 1921.6, 2684.8, 3433.6, 4182.4, 4931.2, 5694.4, 6443.2, 7192]
delivery_latency_OFDMA["MCS 3"] = [985.6, 1547.2, 2108.8, 2670.4, 3246.4, 3808, 4369.6, 4931.2, 5507.2]
delivery_latency_OFDMA["MCS 4"] = [798.4, 1172.8, 1547.2, 1921.6, 2296, 2670.4, 3059.2, 3433.6, 3808]
delivery_latency_OFDMA["MCS 6"] = [668.8, 913.6, 1172.8, 1417.6, 1676.8, 1921.6, 2180.8, 2425.6, \
                                   2670.4, 2929.6, 3174.4, 3433.6, 3678.4, 3937.6, 4182.4, 4427.2, \
                                   4686.4, 4931.2, 5190.4, 5435.2, 5694.4, 5939.2, 6184, 6443.2, \
                                   6688, 6947.2, 7192, 7451.2, 7696, 7940.8, 8200, 8444.8]

In [ ]:
# Set the simulation parameters

results_directory_simulation = "../results/simulation_4/"
config_file = "No config file"

setting_reserved = "setting 0"
setting_contention = "setting 6"
payload_size = {"reserved": parameters[setting_reserved]["payload_size"]*parameters[setting_reserved]["aggregation"], 
                "contention": parameters[setting_contention]["payload_size"]*parameters[setting_contention]["aggregation"]}
delivery_latency = {"reserved": parameters[setting_reserved]["delivery_latency"],
                    "contention": parameters[setting_contention]["delivery_latency"]}
PER = {"reserved":  parameters[setting_reserved]["PER"], 
       "contention":  0}
aggregation_limit = 104


OFDMA_MCS = "MCS 6"
OFDMA_delivery_latency = delivery_latency_OFDMA[OFDMA_MCS]
OFDMA_PER = 0.3
delivery_latency["OFDMA"] = OFDMA_delivery_latency
PER["OFDMA"] = OFDMA_PER


num_UEs = 20
UE_names = ["UE" + str(i) for i in range(num_UEs)]
UE_names = ["UE" + str(i) for i in range(int(num_UEs/2))] + ["UE" + str(i) + "_shadow" for i in range(int(num_UEs/2))]
num_packets_per_ue = None  # Number of packets per UE for the whole period
packet_sizes = [parameters[setting_reserved]["payload_size"]] # TODO: Both have same packet size, but what if they don't?
priorities = [1]
# lambda_range = np.logspace(-4.5, -3, 20)
# lambda_range = np.concatenate((np.logspace(-4.5, -3, 10), np.logspace(-3, -2.2, 5)))
# For 10 UEs
# lambda_range = np.logspace(-4.5, -3.765, 15)
# For aggregation 5
# lambda_range = np.concatenate((np.logspace(-4.5, -3.72, 6), np.logspace(-3.61, -3.43, 9)))
# For aggregation 10
# lambda_range = np.concatenate((np.logspace(-4.5, -3.83, 5), np.logspace(-3.75, -3.35, 10)))
lambda_range = np.concatenate((np.logspace(-4.5, -3.83, 5), np.logspace(-3.75, -2.5, 20))) # testing code speed
# lambda_range = [lambda_range[-1]]
# lambda_range = [3000/10**6]
# For aggregation 2
# lambda_range = np.concatenate((np.logspace(-4.5, -3.8, 5), np.logspace(-3.745, -3.4, 5)))
# lambda_range = [lambda_range[-1]]
# For 3 UEs
# lambda_range = np.logspace(-4.5, -3.26, 15)
# lambda_range = np.concatenate((np.logspace(-4.5, -3.43, 8), np.logspace(-3.34, -3.26, 7)))
# lambda_range = [10**(-4.5)]
# For 5 UEs
# lambda_range = np.logspace(-4.5, -3.45, 10)
# lambda_range = np.concatenate((np.logspace(-4.5, -3.56, 5), np.logspace(-3.5, -3.45, 5)))
# lambda_range = [lambda_range[-1]]
# For 1 UE
# lambda_range = np.logspace(-4.5, -3.5, 10)
lambda_original = copy.deepcopy(lambda_range)
UE_arrival = ["Poisson"]*num_UEs
UE_serve_mode = ["Mode 4"]*num_UEs
num_iterations_arrival = 5
CWmin = 15
CWmax = 1023


## Schedule parameters for reserved base schedule
start_offset = 10 # microseconds
end_time = 1.5*10**6 + start_offset # microseconds
# schedule_config = {"schedule_name": "schedule 5",
#                    "qbv_window_size": 1250,
#                    "num_UEs_together_qbv": 1,
#                    "contention_window_size": 3000,
#                    "num_UEs_together_contention": 2}

# schedule_config = {"schedule_name": "grouped roundrobin",
#                    "qbv_window_size": 2500,
#                    "num_UEs_together": 2,
#                    "offset": 5}

# schedule_config = {"schedule_name": "shuffle roundrobin",
#                    "qbv_window_size": 800,
#                    }

# schedule_config = {"schedule_name": "shuffle roundrobin",
#                               "qbv_window_size": 1500,
#                               "contention_window_size": 1000,
#                               "contention_UE_indices": [4,5,7]}

# schedule_config = {"schedule_name": "roundrobin",
#                      "qbv_window_size": 8700}
                            

# schedule_config = {"schedule_name": "CSMA"}

schedule_config = {"schedule_name": "OFDMA slots",
                   "qbv_window_size": 1500
                   }



# Network properties
# Obtained from the sheet
wifi_slot_time = 9 # microseconds
DIFS = 34 # microseconds



# Plot information
percentile_to_plot = 99
num_iterations_contention = [5]*len(lambda_range)
mode_contention = "Mode 4" 
advance_time = 10 # microseconds
debug_mode = False


In [ ]:
print(delivery_latency["contention"])
dummy_window_length = 800
print("min", bisect.bisect_right(delivery_latency["contention"], dummy_window_length-34-135))
print("max", bisect.bisect_right(delivery_latency["contention"], dummy_window_length-34))
print("average", bisect.bisect_right(delivery_latency["contention"], dummy_window_length-34-67.5))



lambda_value = 700*10**-6
for delivery_latency_index in range(len(delivery_latency["contention"])):
    delivery_latency_value = delivery_latency["contention"][delivery_latency_index]
    slot_length_temp = DIFS + 0 + delivery_latency_value
    if num_UEs*slot_length_temp*lambda_value < delivery_latency_index:
        print(num_UEs*slot_length_temp*lambda_value)
        print(delivery_latency_index)
        print(delivery_latency_value)
        print(slot_length_temp)
        break
    elif delivery_latency_index == len(delivery_latency["contention"])-1:
        print("Not enough")
        print()


In [ ]:
queue_lengths_this_slot = {"UE2": 13, "UE0": 5, "UE6": 10, "UE1": 42}

# Sort the dictionary by values in descending order and extract the keys
sorted_ue_names = sorted(queue_lengths_this_slot, key=queue_lengths_this_slot.get, reverse=True)

print(sorted_ue_names)

In [ ]:
print(num_iterations_contention)
assert len(num_iterations_contention) == len(lambda_range), "Lengths not equal"

In [ ]:
results_directory_simulation = "../results/simulation_4/"

In [ ]:
assert len(lambda_range) == len(num_iterations_contention), "Lengths not equal"
print(lambda_range)

In [ ]:
schedule_contention, cycle_time = create_schedule(UE_names, start_offset, end_time,\
                                      schedule_config)
print(schedule_contention)

from matplotlib.patches import Rectangle

# Plotting the schedule
fig, ax = plt.subplots(figsize=(10, 5))

# Define Y-axis labels and their corresponding positions
ue_positions = {ue: i for i, ue in enumerate(UE_names)}
height = 1  # Height of the rectangles

# Plot rectangles for each slot
for slot in schedule_contention.schedule.values():
    for ue in slot.UEs:
        if slot.mode == "contention":
            rect = Rectangle((slot.start_time, ue_positions[ue] - height / 2), slot.end_time - slot.start_time, height, color='red', alpha=0.8)
        elif slot.mode == "OFDMA":
            rect = Rectangle((slot.start_time, ue_positions[ue] - height / 2), slot.end_time - slot.start_time, height, facecolor='green', alpha=0.8, edgecolor='black', linewidth = 0.5)
        ax.add_patch(rect)
        # ax.text((slot.start_time + slot.end_time) / 2, ue_positions[ue], ue, horizontalalignment='center', verticalalignment='center')
        
        ax.plot([slot.start_time, slot.start_time], [ue_positions[ue] - height / 2, ue_positions[ue] + height / 2], color='black', linewidth=1.6)
        ax.plot([slot.end_time, slot.end_time], [ue_positions[ue] - height / 2, ue_positions[ue] + height / 2], color='black', linewidth=1.6)


# Set Y-axis with UE names
ax.set_yticks(list(ue_positions.values()))
ax.set_yticklabels(UE_names)

# Set labels and title
ax.set_xlabel('Time (microseconds)')
ax.set_title('UE Activity Schedule')

# Set limits for the axes
ax.set_xlim(start_offset, cycle_time)
ax.set_ylim(-1, len(UE_names))
ax.grid(True)

plt.tight_layout()
plt.show()



In [ ]:
# Create UEs and serve the packets, not integrated with result extraction

results_per_lambda_contention = {}

count = 0

execution_start_time = time.time()

for lambda_value in lambda_range:
    
    print("\n###### Lambda value: " + str(lambda_value), ", Count: " + str(count), "######")
    
    
    results_per_lambda_per_iteration_contention = {}
    for num_arrival_iteration in range(num_iterations_arrival):
        print("\nArrival iteration: " + str(num_arrival_iteration))
        # Create UEs and packets
        
        UEs_contention = {}
        
        time_generate_ues_start = time.time()
        for i in range(num_UEs): 
            # TODO: Move the UE creation parameters to the cell above?
            UE_temp = UE(i, {1: 0, 2: 1}, UE_arrival[i], UE_serve_mode[i],  num_packets_per_ue, \
                         CWmin=CWmin, CWmax=CWmax)
            UE_temp.set_poisson_lambda(lambda_value)
            UE_temp.initialize_transmission_record(schedule_contention)
            UE_temp.generate_packets(schedule_contention, packet_sizes, priorities) # TODO: Change this
            UEs_contention[UE_names[i]] = UE_temp
        time_generate_ues_finish = time.time()

        

        # TODO: Check that the delivery times are always in ascending order
        # TODO: check that the arrival times are always in ascending order

        # TODO: Make this more general i.e handle packet statuses directly instead of opearting under the 
        # restrictions of this simulation
        print("Num packets: " + str(UEs_contention["UE0"].n_packets))


        # Serve the packets with contention
        results_iteration = {}
        

        for i in range(num_iterations_contention[count]):
            print("Contention iteration: " + str(i))
            time_deep_copy_ues_start = time.time()
            UEs_contention_temp = copy.deepcopy(UEs_contention)
            time_deep_copy_ues_finish = time.time()

            print("Generate test network")
            test_network = Network(wifi_slot_time, DIFS, UEs_contention_temp, debug_mode)

            time_serve_packets_start = time.time()  
            # %lprun -f Network.serve_packets test_network.serve_packets(schedule_contention,\
            #                                     mode_contention, \
            #                                     payload_size = payload_size, \
            #                                     delivery_latency = delivery_latency, \
            #                                     PER = PER, advance_time = advance_time, \
            #                                     aggregation_limit = aggregation_limit)
            test_network.serve_packets(schedule_contention,\
                                                mode_contention, \
                                                payload_size = payload_size, \
                                                delivery_latency = delivery_latency, \
                                                PER = PER, advance_time = advance_time, \
                                                aggregation_limit = aggregation_limit)
            time_serve_packets_finish = time.time()

            # TODO: remove this
            # print("Base schedule:", schedule_contention)
            

            print("Save UEs_contetion_temp")
            results_iteration[i] = UEs_contention_temp 
        # for key in results_iteration:
        #     print("results_iteration " + str(key), results_iteration[key])

        # TODO: Scale to multiple UEs, currently you're extracting the results only for one UE,
        # but you should be extracting the results for all UEs
        
        print("Save results_iteration")
        results_per_lambda_per_iteration_contention[num_arrival_iteration] = results_iteration
    
    print("Save results_per_lambda_per_iteration_contention")
    %time results_per_lambda_contention[lambda_value] = results_per_lambda_per_iteration_contention
    count = count + 1
    

execution_finish_time = time.time()
execution_duration = execution_finish_time - execution_start_time


print("Execution duration: " + str(execution_duration))
print("Generate ues duration: " + str(time_generate_ues_finish - time_generate_ues_start))
print("Serve packets duration: " + str(time_serve_packets_finish - time_serve_packets_start))
print("Deep copy ues duration: " + str(time_deep_copy_ues_finish - time_deep_copy_ues_start))

In [ ]:
# find slots that UEs are allowed to transmit in and print UEs
UE_to_times = {}
for UE_name in UE_names:
    UE_to_times[UE_name] = []

print(UE_to_times)

for slot in schedule_contention.schedule:
    for UE_name in schedule_contention.schedule[slot].UEs:
        UE_to_times[UE_name].append((schedule_contention.schedule[slot].start_time, schedule_contention.schedule[slot].end_time))

print(UE_to_times)
# del UE_to_times["UE0"][-1]
for ue in UEs_contention_temp:
    print("UE: " + str(ue))
    print(UEs_contention_temp[ue])
    # for packet in UEs_contention_temp[ue].packets:
    #     if packet.delivery_time is not None:
    #         assert packet.arrival_time < packet.delivery_time
    #         correct_slot_delivery = False
    #         for slot_times in UE_to_times[ue]:
    #             if packet.delivery_time >= slot_times[0] and packet.delivery_time <= slot_times[1]:
    #                 correct_slot_delivery = True
    #                 break
            # assert correct_slot_delivery, "Delivery time not in correct slot : " + str(packet.sequence_number) + " " + str(packet.delivery_time)

In [ ]:
# parse the results


results_allUEs_per_lambda_contention = {}
for lambda_value in results_per_lambda_contention:
    print("\n\nlambda value: ", lambda_value)

    mean_latencies_across_arrivals = []
    percentile_latencies_across_arrivals = []
    n_packets_not_served_across_arrivals = []
    contention_wins_across_arrivals = []
    bus_occupancy_across_arrivals = []
    queue_slope_across_arrivals = []

    for num_iteration_arrival in [0]:
        mean_latencies = []
        percentile_latencies = []
        n_packets_not_served_array = []
        contention_wins = []
        bus_occupancy = []
        queue_slope = []
        print("arrival iteration " + str(num_iteration_arrival))
        for iteration in results_per_lambda_contention[lambda_value][num_iteration_arrival]:
            latencies = []
            bus_occupancy_across_ues = []
            contention_wins_across_ues = []
            queue_slope_across_ues = []
            n_packets_not_served = 0
            # print("iteration", iteration)
            for ue in results_per_lambda_contention[lambda_value][num_iteration_arrival][iteration]:
                print("UE: ", ue)
                UE_temp = results_per_lambda_contention[lambda_value][num_iteration_arrival][iteration][ue]
                print(UE_temp)
                latencies_UE = UE_temp.obtain_packet_latency()
                print(latencies_UE)
                latencies_UE = [latency for latency in latencies_UE if latency is not None]
                n_packets_not_served += UE_temp.n_packets - len(latencies_UE)
                latencies.extend(latencies_UE)
                
                contention_wins_across_ues.append(UE_temp.transmission_record[0]["num_wins"])
                bus_occupancy_across_ues.append(np.mean(UE_temp.transmission_record[0]["num_transmissions"]))
                # print(bus_occupancy_across_ues)
                
                queue_lengths = []
                queue_times = []
                for slot in UE_temp.transmission_record:
                    queue_lengths.extend(UE_temp.transmission_record[slot]["queue_information"]["queue_lengths"])
                    queue_times.extend(UE_temp.transmission_record[slot]["queue_information"]["queue_times"])

                # queue_lengths = np.array(UE_temp.transmission_record[0]["queue_information"]["queue_lengths"])
                # queue_times = np.array(UE_temp.transmission_record[0]["queue_information"]["queue_times"])
                # TODO: add a scaling factor
                # queue_slopes = (queue_lengths[1:] - queue_lengths[:-1])/(queue_times[1:] - queue_times[:-1])
                # queue_slope_across_ues.append(np.mean(queue_slopes))
                slope, intercept = np.polyfit(queue_times, queue_lengths, 1)
                queue_slope_across_ues.append(slope)


            print("iteration", iteration)    
            mean_latencies.append(np.mean(latencies))
            percentile_latencies.append(compute_percentile(latencies, percentile_to_plot))
            n_packets_not_served_array.append(n_packets_not_served)
            contention_wins.append(np.mean(contention_wins_across_ues))
            bus_occupancy.append(np.mean(bus_occupancy_across_ues))
            queue_slope.append(np.mean(queue_slope_across_ues))

        print("Len(mean_latencies)", len(mean_latencies))
        mean_latencies_across_arrivals.append(np.mean(mean_latencies))
        percentile_latencies_across_arrivals.append(np.mean(percentile_latencies))
        n_packets_not_served_across_arrivals.append(np.mean(n_packets_not_served_array))
        contention_wins_across_arrivals.append(np.mean(contention_wins))
        bus_occupancy_across_arrivals.append(np.mean(bus_occupancy))
        queue_slope_across_arrivals.append(np.mean(queue_slope))

    result_temp = {}        
    result_temp["mean_latency"] = np.mean(mean_latencies_across_arrivals)
    print("mean_latencies_across_arrivals", mean_latencies_across_arrivals)
    
    result_temp["mean_latency_std"] = np.std(mean_latencies_across_arrivals)
    result_temp["percentile_latency"] = np.mean(percentile_latencies_across_arrivals)
    result_temp["percentile_latency_std"] = np.std(percentile_latencies_across_arrivals)
    result_temp["n_packets_not_served"] = np.mean(n_packets_not_served_across_arrivals)
    result_temp["n_packets_not_served_std"] = np.std(n_packets_not_served_across_arrivals)
    result_temp["contention_wins"] = np.mean(contention_wins_across_arrivals)
    result_temp["bus_occupancy"] = np.mean(bus_occupancy_across_arrivals)
    result_temp["queue_slope"] = np.mean(queue_slope_across_arrivals)
    results_allUEs_per_lambda_contention[lambda_value] = result_temp

In [ ]:
UE_name = "UE3"

x = []
y = []
UE_queue_plot = results_per_lambda_contention[lambda_range[0]][0][0]["UE0"]
for slot in UE_queue_plot.transmission_record:
    print("Slot: " + str(slot))
    y += UE_queue_plot.transmission_record[slot]["queue_information"]["queue_lengths"]
    print(UE_queue_plot.transmission_record[slot]["queue_information"]["queue_lengths"])
    x += UE_queue_plot.transmission_record[slot]["queue_information"]["queue_times"]
slope, intercept = np.polyfit(x, y, 1)
plt.title("Queue length vs time")
plt.xlabel("Time")
plt.ylabel("Queue length")
plt.plot(x, y, label = "queue information")
plt.plot(x, slope*np.array(x) + intercept, label = "best fit line")
print(slope)
print(x)

In [ ]:
# Create a results directory folder using results_directory_simulation and the current time
experiment_folder_name = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
results_directory_experiment = os.path.join(results_directory_simulation, experiment_folder_name)
os.makedirs(results_directory_experiment, exist_ok=True)
# Plots: CDF of latencies, percentile latency vs lambda, mean latency vs lambda,
# number of packets not served vs lambda

In [ ]:
# parse the results


results_allUEs_per_lambda_contention = {}
for lambda_value in results_per_lambda_contention:
    print("\n\nlambda value: ", lambda_value)

    mean_latencies_across_arrivals = []
    percentile_latencies_across_arrivals = []
    n_packets_not_served_across_arrivals = []
    contention_wins_across_arrivals = []
    bus_occupancy_across_arrivals = []
    queue_slope_across_arrivals = []

    for num_iteration_arrival in results_per_lambda_contention[lambda_value]:
        mean_latencies = []
        percentile_latencies = []
        n_packets_not_served_array = []
        contention_wins = []
        bus_occupancy = []
        queue_slope = []
        print("arrival iteration " + str(num_iteration_arrival))
        for iteration in results_per_lambda_contention[lambda_value][num_iteration_arrival]:
            latencies = []
            bus_occupancy_across_ues = []
            contention_wins_across_ues = []
            queue_slope_across_ues = []
            n_packets_not_served = 0
            # print("iteration", iteration)
            for ue in results_per_lambda_contention[lambda_value][num_iteration_arrival][iteration]:
                # print("UE: ", ue)
                UE_temp = results_per_lambda_contention[lambda_value][num_iteration_arrival][iteration][ue]
                latencies_UE = UE_temp.obtain_packet_latency()
                latencies_UE = [latency for latency in latencies_UE if latency is not None]
                n_packets_not_served += UE_temp.n_packets - len(latencies_UE)
                latencies.extend(latencies_UE)
                
                contention_wins_across_ues.append(UE_temp.transmission_record[0]["num_wins"])
                bus_occupancy_across_ues.append(np.mean(UE_temp.transmission_record[0]["num_transmissions"]))
                # print(bus_occupancy_across_ues)
                
                queue_lengths = []
                queue_times = []
                for slot in UE_temp.transmission_record:
                    queue_lengths.extend(UE_temp.transmission_record[slot]["queue_information"]["queue_lengths"])
                    queue_times.extend(UE_temp.transmission_record[slot]["queue_information"]["queue_times"])

                # queue_lengths = np.array(UE_temp.transmission_record[0]["queue_information"]["queue_lengths"])
                # queue_times = np.array(UE_temp.transmission_record[0]["queue_information"]["queue_times"])
                # TODO: add a scaling factor
                # queue_slopes = (queue_lengths[1:] - queue_lengths[:-1])/(queue_times[1:] - queue_times[:-1])
                # queue_slope_across_ues.append(np.mean(queue_slopes))
                slope, intercept = np.polyfit(queue_times, queue_lengths, 1)
                queue_slope_across_ues.append(slope)


            print("iteration", iteration)    
            mean_latencies.append(np.mean(latencies))
            percentile_latencies.append(compute_percentile(latencies, percentile_to_plot))
            n_packets_not_served_array.append(n_packets_not_served)
            contention_wins.append(np.mean(contention_wins_across_ues))
            bus_occupancy.append(np.mean(bus_occupancy_across_ues))
            queue_slope.append(np.mean(queue_slope_across_ues))

        print("Len(mean_latencies)", len(mean_latencies))
        mean_latencies_across_arrivals.append(np.mean(mean_latencies))
        percentile_latencies_across_arrivals.append(np.mean(percentile_latencies))
        n_packets_not_served_across_arrivals.append(np.mean(n_packets_not_served_array))
        contention_wins_across_arrivals.append(np.mean(contention_wins))
        bus_occupancy_across_arrivals.append(np.mean(bus_occupancy))
        queue_slope_across_arrivals.append(np.mean(queue_slope))

    result_temp = {}        
    result_temp["mean_latency"] = np.mean(mean_latencies_across_arrivals)
    print("mean_latencies_across_arrivals", mean_latencies_across_arrivals)
    
    result_temp["mean_latency_std"] = np.std(mean_latencies_across_arrivals)
    result_temp["percentile_latency"] = np.mean(percentile_latencies_across_arrivals)
    result_temp["percentile_latency_std"] = np.std(percentile_latencies_across_arrivals)
    result_temp["n_packets_not_served"] = np.mean(n_packets_not_served_across_arrivals)
    result_temp["n_packets_not_served_std"] = np.std(n_packets_not_served_across_arrivals)
    result_temp["contention_wins"] = np.mean(contention_wins_across_arrivals)
    result_temp["bus_occupancy"] = np.mean(bus_occupancy_across_arrivals)
    result_temp["queue_slope"] = np.mean(queue_slope_across_arrivals)
    results_allUEs_per_lambda_contention[lambda_value] = result_temp

In [ ]:
# Save the parameters and the results of the experiment to a file
experiment_parameters = {
        "config_file": config_file,
        "setting_reserved": parameters[setting_reserved],
        "setting_contention": parameters[setting_contention],
        "aggregation_limit": aggregation_limit,
        "schedule_config": schedule_config,
        "num_UEs": num_UEs,
        "num_packets_per_ue": num_packets_per_ue,
        "packet_sizes": packet_sizes,
        "priorities": priorities,
        "UE_arrival": UE_arrival,
        "UE_serve_mode": UE_serve_mode,
        "start_offset": start_offset, # microseconds
        "end_time": end_time,
        "percentile_to_plot": percentile_to_plot,
        "wifi_slot_time": wifi_slot_time,
        "DIFS": DIFS,
        "num_iterations_contention": num_iterations_contention,
        "num_iterations_arrival": num_iterations_arrival,
        "contention_mode": mode_contention,
        "advance_time": advance_time,
        "CWmin": CWmin,
        "CWmax": CWmax,
        "lambda_range": lambda_range,
        "execution_duration": execution_duration,
    }
# Write experiment_parameters_json to a json file with filename experiment_parameters.json

experiment_parameters_json = json.dumps(experiment_parameters, indent=4, cls=NumpyEncoder)
experiment_parameters_json_filename = os.path.join(results_directory_experiment, \
                                                   "experiment_parameters.json")
with open(experiment_parameters_json_filename, "w") as file:
    file.write(experiment_parameters_json)

latencies = UEs_contention_temp["UE0"].obtain_packet_latency()
experiment_parameters_pickle = {
        "schedule_contention": schedule_contention,
        "results_allUEs_per_lambda_contention": results_allUEs_per_lambda_contention,
        "experiment_parameters": experiment_parameters
    }

experiment_parameters_pickle_filename = os.path.join(results_directory_experiment, \
                                                    "experiment_parameters.pkl")

with open(experiment_parameters_pickle_filename, "wb") as file:
    pickle.dump(experiment_parameters_pickle, file)

In [ ]:
# Plot the results

lambda_range = lambda_original

# lambda_range = lambda_range[:8]

scale = "linear"
percentile_filename = "percentile_latency_allUEs_all_" + scale + ".png"
percentile_slope_filename = "percentile_slope_allUEs_all_" + scale + ".png"
mean_filename = "mean_latency_allUEs_all_" + scale + ".png"
mean_slope_filename = "mean_slope_allUEs_all_" + scale + ".png"
n_packets_not_served_filename = "n_packets_not_served_allUEs_all_" + scale + ".png"
bus_occupancy_filename = "bus_occupancy_10UEs_all_extended_" + scale + ".png"
n_wins_filename = "n_wins_10UEs_all_extended_" + scale + ".png"
scaling_factor = 1e6

# Plot the percentile curve

plt.figure(figsize=(10, 8))
# percentiles = []
# for lambda_value in lambda_range:
#     percentiles.append(results_allUEs_per_lambda_reserved[lambda_value]["percentile_latency"])
# plt.plot(np.array(lambda_range)*(schedule_reserved.end_time - schedule_reserved.start_time), \
#          percentiles, ".-", label = "reserved")

percentiles_contention = []
percentiles_contention_std = []
for lambda_value in lambda_range:
    percentiles_contention.append(results_allUEs_per_lambda_contention[lambda_value]["percentile_latency"])
    percentiles_contention_std.append(\
        results_allUEs_per_lambda_contention[lambda_value]["percentile_latency_std"])
plt.errorbar(np.array(lambda_range)*scaling_factor, \
        percentiles_contention, percentiles_contention_std, label = "contention", fmt='.-', \
        capsize=3)
# plt.plot(n_packets_generated, percentiles)
plt.xlabel("lambda (packets/s)")
plt.ylabel(str(percentile_to_plot) + "percentile latency (us)")
plt.legend()
plt.grid()

if scale == "log":
        plt.yscale('log')

title = (f"Simulation 3 {percentile_to_plot} percentile latency vs lambda, \n PER = {PER},\n"
         f"num_UEs: {num_UEs}, \n"
         f"allowed_payload: {payload_size} B, \n "
         f"packet size: {packet_sizes[0]} B, \n"
        )
plt.title(title)
# Insert a textbox at the lowest y value of the plot and have y axis be the label
plt.text(0, percentiles_contention[0], str(np.round(percentiles_contention[0],2)), \
         fontsize=12, verticalalignment='bottom')

plt.axhline(y=5000, color='r', linestyle='--')
plt.ylim(0,100000)
# plt.xlim(0, 5000)


plt.tight_layout()




plt.savefig(os.path.join(results_directory_experiment, percentile_filename))
plt.show()


slope = np.diff(percentiles_contention)/(np.diff(lambda_range)*(schedule_contention.end_time - schedule_contention.start_time))
plt.title("Percentile latency slope")
plt.xlabel("lambda*schedule_duration (us)")
plt.ylabel(str(percentile_to_plot) + "percentile latency slope (us)")
if scale == "log":
        plt.yscale('log')
        plt.ylim(10**-2, 10**2)
plt.plot(np.array(lambda_range[1:])*(schedule_contention.end_time - schedule_contention.start_time), slope, ".-")
plt.savefig(os.path.join(results_directory_experiment, percentile_slope_filename))

print(slope)
# Plot the mean latency curve

plt.figure(figsize=(10, 8))
# mean_latencies = []
# for lambda_value in lambda_range:
#     mean_latencies.append(results_allUEs_per_lambda_reserved[lambda_value]["mean_latency"])
# plt.plot(np.array(lambda_range)*(schedule_reserved.end_time - schedule_reserved.start_time),\
#          mean_latencies, ".-", label = "reserved")



mean_latencies_contention = []
mean_latencies_contention_std = []
for lambda_value in lambda_range:
    mean_latencies_contention.append(results_allUEs_per_lambda_contention[lambda_value]["mean_latency"])
    mean_latencies_contention_std.append(\
        results_allUEs_per_lambda_contention[lambda_value]["mean_latency_std"])
plt.errorbar(np.array(lambda_range)*scaling_factor,\
        mean_latencies_contention, mean_latencies_contention_std, label = "contention", fmt='.-', \
        capsize=3)


plt.text(0, mean_latencies_contention[0], str(np.round(mean_latencies_contention[0],2)), \
         fontsize=12, verticalalignment='bottom')

plt.axhline(y=5000, color='r', linestyle='--')
plt.ylim(0,15000) 
# plt.xlim(0, 5000)

plt.legend()
plt.grid()

plt.xlabel("lambda (packets/s)")
plt.ylabel("Mean latency (us)")

if scale == "log":
        plt.yscale('log')

title = (f"Simulation 3 mean latency vs lambda, \n PER = {PER}, \n"
         f"num_UEs: {num_UEs}, \n"
         f"allowed_payload: {payload_size} B, \n "
         f"packet size: {packet_sizes[0]} B, \n"
         )
plt.title(title)
plt.tight_layout()

plt.savefig(os.path.join(results_directory_experiment, mean_filename))


plt.show()


slope = np.diff(mean_latencies_contention)/(np.diff(lambda_range)*(schedule_contention.end_time - schedule_contention.start_time))
plt.title("Mean latency slope")
plt.xlabel("lambda*schedule_duration (us)")
plt.ylabel("Mean percentile latency slope (us)")
if scale == "log":
        plt.yscale('log')
plt.plot(np.array(lambda_range[1:])*(schedule_contention.end_time - schedule_contention.start_time), slope, ".-")
plt.savefig(os.path.join(results_directory_experiment, mean_slope_filename))

plt.figure(figsize=(10, 8))
# mean_latencies = []
# for lambda_value in lambda_range:
#     mean_latencies.append(results_allUEs_per_lambda_reserved[lambda_value]["mean_latency"])
# plt.plot(np.array(lambda_range)*(schedule_reserved.end_time - schedule_reserved.start_time),\
#          mean_latencies, ".-", label = "reserved")

# scale = "linear"


unserved_packets_contention = []
unserved_packets_contention_std = []
for lambda_value in lambda_range:
    unserved_packets_contention.append(results_allUEs_per_lambda_contention[lambda_value]["n_packets_not_served"])
    unserved_packets_contention_std.append(\
        results_allUEs_per_lambda_contention[lambda_value]["n_packets_not_served_std"])
plt.errorbar(np.array(lambda_range)*scaling_factor,\
        np.array(unserved_packets_contention) + 1, np.array(unserved_packets_contention_std) + 0.001, label = "contention", fmt='.-', \
        capsize=3)


# plt.text(0, unserved_packets_contention[0], str(np.round(unserved_packets_contention[0],2)), \
        #  fontsize=12, verticalalignment='bottom')

plt.legend()

plt.xlabel("lambda (packets/s)")
plt.ylabel("Unserved packets")

if scale == "log":
        plt.yscale('log')

title = (f"Simulation 3 unserved vs lambda, \n PER = {PER}, \n"
         f"num_UEs: {num_UEs}, \n"
         f"allowed_payload: {payload_size} B, \n "
         f"packet size: {packet_sizes[0]} B, \n"
        )
plt.title(title)
plt.tight_layout()

plt.savefig(os.path.join(results_directory_experiment, n_packets_not_served_filename))

plt.show()




bus_occupancy_contention = []
for lambda_value in lambda_range:
    bus_occupancy_contention.append(results_allUEs_per_lambda_contention[lambda_value]["bus_occupancy"])
plt.plot(np.array(lambda_range)*scaling_factor, \
        bus_occupancy_contention, '.-', label = "contention")
# plt.plot(n_packets_generated, percentiles)
plt.xlabel("lambda (packets/s)")
plt.ylabel("Bus occupancy")
plt.legend()

if scale == "log":
        plt.yscale('log')

title = (f"Simulation 3 Bus occupancy vs lambda, \n PER = {PER},\n"
         f"num_UEs: {num_UEs}, \n"
         f"allowed_payload: {payload_size} B, \n "
         f"packet size: {packet_sizes[0]} B, \n"
        )
plt.title(title)
# Insert a textbox at the lowest y value of the plot and have y axis be the label
plt.tight_layout()
plt.savefig(os.path.join(results_directory_experiment, bus_occupancy_filename))
plt.show()


wins_contention = []
for lambda_value in lambda_range:
    wins_contention.append(results_allUEs_per_lambda_contention[lambda_value]["contention_wins"])
plt.plot(np.array(lambda_range)*scaling_factor, \
        wins_contention, '.-', label = "contention")
# plt.plot(n_packets_generated, percentiles)
plt.xlabel("lambda (packets/s)")
plt.ylabel("Contention wins")
plt.legend()

if scale == "log":
        plt.yscale('log')

title = (f"Simulation 3 num_wins vs lambda, \n PER = {PER},\n"
         f"num_UEs: {num_UEs}, \n"
         f"allowed_payload: {payload_size} B, \n "
         f"packet size: {packet_sizes[0]} B, \n"
        )
plt.title(title)
# Insert a textbox at the lowest y value of the plot and have y axis be the label
plt.tight_layout()
plt.savefig(os.path.join(results_directory_experiment, n_wins_filename))
plt.show()


queue_slope = []
for lambda_value in lambda_range:
    queue_slope.append(results_allUEs_per_lambda_contention[lambda_value]["queue_slope"])
plt.plot(np.array(lambda_range)*scaling_factor, \
        queue_slope, '.-', label = "contention")
# plt.plot(n_packets_generated, percentiles)
plt.xlabel("lambda (packets/s)")
plt.ylabel("queue slope")
plt.legend()

if scale == "log":
        plt.yscale('log')

title = (f"Simulation 3 queue slope vs lambda, \n PER = {PER},\n"
         f"num_UEs: {num_UEs}, \n"
         f"allowed_payload: {payload_size} B, \n "
         f"packet size: {packet_sizes[0]} B, \n"
        )
plt.title(title)
# Insert a textbox at the lowest y value of the plot and have y axis be the label
plt.tight_layout()
plt.savefig(os.path.join(results_directory_experiment, n_wins_filename))
plt.show()

In [ ]:
print(bus_occupancy_contention)
print(mean_latencies_contention)
print(percentiles_contention)

# Integrating result extraction into the main body of the code 

In [ ]:
# # Create a results directory folder using results_directory_simulation and the current time
# experiment_folder_name = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
# results_directory_experiment = os.path.join(results_directory_simulation, experiment_folder_name)
# UEs_directory = os.path.join(results_directory_experiment, "UEs")
# os.makedirs(results_directory_experiment, exist_ok=True)
# os.makedirs(UEs_directory, exist_ok=True)
# # Plots: CDF of latencies, percentile latency vs lambda, mean latency vs lambda,
# # number of packets not served vs lambda

In [ ]:
# # Create a schedule, UEs and serve the packets, not integrated with result extraction

# slots_temp = {}
# slots_temp[0] = Slot(0, start_offset, end_time, "contention", UE_names)
# schedule_contention = Schedule(start_offset, end_time, 1, slots_temp)


# # print(schedule_reserved)
# print(schedule_contention)

# results_per_lambda_contention = {}

# count = 0

# execution_start_time = time.time()


# results_allUEs_per_lambda_contention = {}

# for lambda_value in lambda_range:
    
#     print("\n###### Lambda value: " + str(lambda_value), ", Count: " + str(count), "######")
    
#     mean_latencies_across_arrivals = []
#     percentile_latencies_across_arrivals = []
#     n_packets_not_served_across_arrivals = []
#     contention_wins_across_arrivals = []
#     bus_occupancy_across_arrivals = []
    
#     results_per_lambda_per_iteration_contention = {}
#     for num_arrival_iteration in range(num_iterations_arrival):
#         print("\nArrival iteration: " + str(num_arrival_iteration))
#         # Create UEs and packets
        
#         UEs_contention = {}
        
        
#         time_generate_ues_start = time.time()
#         for i in range(num_UEs): 
#             # TODO: Move the UE creation parameters to the cell above?
#             UE_temp = UE(i, {1: 0, 2: 1}, UE_arrival[i], UE_serve_mode[i],  num_packets_per_ue, \
#                          CWmin=CWmin, CWmax=CWmax)
#             UE_temp.set_poisson_lambda(lambda_value)
#             UE_temp.initialize_transmission_record(schedule_contention)
#             UE_temp.generate_packets(schedule_contention, packet_sizes, priorities) # TODO: Change this
#             UEs_contention[UE_names[i]] = UE_temp
#         time_generate_ues_finish = time.time()

#         # TODO: Check that the delivery times are always in ascending order
#         # TODO: check that the arrival times are always in ascending order

#         # TODO: Make this more general i.e handle packet statuses directly instead of opearting under the 
#         # restrictions of this simulation
#         print("Num packets: " + str(UEs_contention["UE0"].n_packets))


#         # Serve the packets with contention
#         results_iteration = {}
#         mean_latencies = []
#         percentile_latencies = []
#         n_packets_not_served_array = []
#         contention_wins = []
#         bus_occupancy = []

#         for i in range(num_iterations_contention[count]):
#             print("Contention iteration: " + str(i))
#             time_deep_copy_ues_start = time.time()
#             UEs_contention_temp = copy.deepcopy(UEs_contention)
#             time_deep_copy_ues_finish = time.time()

#             print("Generate test network")
#             test_network = Network(wifi_slot_time, DIFS, UEs_contention_temp, debug_mode)


            

#             time_serve_packets_start = time.time()  
#             # %lprun -f Network.serve_packets test_network.serve_packets(schedule_contention,\
#             #                                     mode_contention, \
#             #                                     payload_size = payload_size, \
#             #                                     delivery_latency = delivery_latency, \
#             #                                     PER = PER, advance_time = advance_time)
#             test_network.serve_packets(schedule_contention,\
#                                                 mode_contention, \
#                                                 payload_size = payload_size, \
#                                                 delivery_latency = delivery_latency, \
#                                                 PER = PER, advance_time = advance_time)
#             time_serve_packets_finish = time.time()

#             latencies = []
#             bus_occupancy_across_ues = []
#             contention_wins_across_ues = []
#             n_packets_not_served = 0


#             for ue in UEs_contention_temp:
#                 # print("UE: ", ue)
#                 UE_temp = UEs_contention_temp[ue]
#                 latencies_UE = UE_temp.obtain_packet_latency()
#                 latencies_UE = [latency for latency in latencies_UE if latency is not None]
#                 n_packets_not_served += UE_temp.n_packets - len(latencies_UE)
#                 latencies.extend(latencies_UE)
#                 contention_wins_across_ues.append(UE_temp.transmission_record[0]["num_wins"])
#                 bus_occupancy_across_ues.append(np.mean(UE_temp.transmission_record[0]["num_transmissions"]))
            
#             mean_latencies.append(np.mean(latencies))
#             percentile_latencies.append(compute_percentile(latencies, percentile_to_plot))
#             n_packets_not_served_array.append(n_packets_not_served)
#             contention_wins.append(np.mean(contention_wins_across_ues))
#             bus_occupancy.append(np.mean(bus_occupancy_across_ues))

#             print("Save UEs_contetion_temp")
#             # results_iteration[i] = UEs_contention_temp
#             UEs_filename = os.path.join(UEs_directory, "UEs_contention_" + \
#                                         str(count) + "_" + str(num_arrival_iteration) + "_" + \
#                                         str(i) + ".pkl")
#             with open(UEs_filename, "wb") as file:
#                 pickle.dump(UEs_contention_temp, file) 
        
#         mean_latencies_across_arrivals.append(np.mean(mean_latencies))
#         percentile_latencies_across_arrivals.append(np.mean(percentile_latencies))
#         n_packets_not_served_across_arrivals.append(np.mean(n_packets_not_served_array))
#         contention_wins_across_arrivals.append(np.mean(contention_wins))
#         bus_occupancy_across_arrivals.append(np.mean(bus_occupancy))
        
#         # for key in results_iteration:
#         #     print("results_iteration " + str(key), results_iteration[key])

#         # TODO: Scale to multiple UEs, currently you're extracting the results only for one UE,
#         # but you should be extracting the results for all UEs
        
#         print("Save results_iteration")
#         # results_per_lambda_per_iteration_contention[num_arrival_iteration] = results_iteration
    
#     result_temp = {}        
#     result_temp["mean_latency"] = np.mean(mean_latencies_across_arrivals)
#     result_temp["mean_latency_std"] = np.std(mean_latencies_across_arrivals)
#     result_temp["percentile_latency"] = np.mean(percentile_latencies_across_arrivals)
#     result_temp["percentile_latency_std"] = np.std(percentile_latencies_across_arrivals)
#     result_temp["n_packets_not_served"] = np.mean(n_packets_not_served_across_arrivals)
#     result_temp["n_packets_not_served_std"] = np.std(n_packets_not_served_across_arrivals)
#     result_temp["contention_wins"] = np.mean(contention_wins_across_arrivals)
#     result_temp["bus_occupancy"] = np.mean(bus_occupancy_across_arrivals)
#     results_allUEs_per_lambda_contention[lambda_value] = result_temp
    
#     print("Save results_per_lambda_per_iteration_contention")
#     results_per_lambda_contention[lambda_value] = results_per_lambda_per_iteration_contention
#     count = count + 1
    

# execution_finish_time = time.time()
# execution_duration = execution_finish_time - execution_start_time


# print("Execution duration: " + str(execution_duration))
# print("Generate ues duration: " + str(time_generate_ues_finish - time_generate_ues_start))
# print("Serve packets duration: " + str(time_serve_packets_finish - time_serve_packets_start))
# print("Deep copy ues duration: " + str(time_deep_copy_ues_finish - time_deep_copy_ues_start))

In [ ]:
# # Create a results directory folder using results_directory_simulation and the current time
# experiment_folder_name = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
# results_directory_experiment = os.path.join(results_directory_simulation, experiment_folder_name)
# os.makedirs(results_directory_experiment, exist_ok=True)
# # Plots: CDF of latencies, percentile latency vs lambda, mean latency vs lambda,
# # number of packets not served vs lambda

In [ ]:
# # Save the parameters and the results of the experiment to a file

# experiment_parameters = {
#     "config_file": config_file,
#     "setting_reserved": parameters[setting_reserved],
#     "setting_contention": parameters[setting_contention],
#     "num_UEs": num_UEs,
#     "num_packets_per_ue": num_packets_per_ue,
#     "packet_sizes": packet_sizes,
#     "priorities": priorities,
#     "UE_arrival": UE_arrival,
#     "UE_serve_mode": UE_serve_mode,
#     "start_offset": start_offset, # microseconds
#     "end_time": end_time,
#     "percentile_to_plot": percentile_to_plot,
#     "wifi_slot_time": wifi_slot_time,
#     "DIFS": DIFS,
#     "num_iterations_contention": num_iterations_contention,
#     "num_iterations_arrival": num_iterations_arrival,
#     "contention_mode": mode_contention,
#     "advance_time": advance_time,
#     "CWmin": CWmin,
#     "CWmax": CWmax,
#     "lambda_range": lambda_range,
#     "execution_duration": execution_duration
# }

# # Write experiment_parameters_json to a json file with filename experiment_parameters.json

# experiment_parameters_json = json.dumps(experiment_parameters, indent=4, cls=NumpyEncoder)
# experiment_parameters_json_filename = os.path.join(results_directory_experiment, \
#                                                    "experiment_parameters.json")
# with open(experiment_parameters_json_filename, "w") as file:
#     file.write(experiment_parameters_json)

# latencies = UEs_contention_temp["UE0"].obtain_packet_latency()
# experiment_parameters_pickle = {
#     # "schedule_contention": schedule_contention,
#     # "results_per_lambda_contention": results_per_lambda_contention,
#     # "results_allUEs_per_lambda_reserved": results_per_lambda_per_iteration_contention,
#     "UEs_contention": mean_latencies_across_arrivals,
#     # "results_allUEs_per_lambda_contention": results_allUEs_per_lambda_contention,
#     # "experiment_parameters": experiment_parameters
# }

# experiment_parameters_pickle_filename = os.path.join(results_directory_experiment, \
#                                                     "experiment_parameters.pkl")

# with open(experiment_parameters_pickle_filename, "wb") as file:
#     pickle.dump(experiment_parameters_pickle, file)

In [ ]:
# # Plot the results

# lambda_range = lambda_original

# # lambda_range = lambda_range[:8]

# scale = "linear"
# percentile_filename = "percentile_latency_allUEs_all_" + scale + ".png"
# percentile_slope_filename = "percentile_slope_allUEs_all_" + scale + ".png"
# mean_filename = "mean_latency_allUEs_all_" + scale + ".png"
# mean_slope_filename = "mean_slope_allUEs_all_" + scale + ".png"
# n_packets_not_served_filename = "n_packets_not_served_allUEs_all_" + scale + ".png"
# bus_occupancy_filename = "bus_occupancy_10UEs_all_extended_" + scale + ".png"
# n_wins_filename = "n_wins_10UEs_all_extended_" + scale + ".png"
# scaling_factor = 1e6

# # Plot the percentile curve

# plt.figure(figsize=(10, 8))
# # percentiles = []
# # for lambda_value in lambda_range:
# #     percentiles.append(results_allUEs_per_lambda_reserved[lambda_value]["percentile_latency"])
# # plt.plot(np.array(lambda_range)*(schedule_reserved.end_time - schedule_reserved.start_time), \
# #          percentiles, ".-", label = "reserved")

# percentiles_contention = []
# percentiles_contention_std = []
# for lambda_value in lambda_range:
#     percentiles_contention.append(results_allUEs_per_lambda_contention[lambda_value]["percentile_latency"])
#     percentiles_contention_std.append(\
#         results_allUEs_per_lambda_contention[lambda_value]["percentile_latency_std"])
# plt.errorbar(np.array(lambda_range)*scaling_factor, \
#         percentiles_contention, percentiles_contention_std, label = "contention", fmt='.-', \
#         capsize=3)
# # plt.plot(n_packets_generated, percentiles)
# plt.xlabel("lambda (packets/s)")
# plt.ylabel(str(percentile_to_plot) + "percentile latency (us)")
# plt.legend()

# if scale == "log":
#         plt.yscale('log')

# title = (f"Simulation 3 {percentile_to_plot} percentile latency vs lambda, \n PER = {PER},\n"
#          f"num_UEs: {num_UEs}, \n"
#          f"allowed_payload: {payload_size} B, \n "
#          f"packet size: {packet_sizes[0]} B, \n"
#          f"delivery_latency: {delivery_latency} us ,\n"
#         )
# plt.title(title)
# # Insert a textbox at the lowest y value of the plot and have y axis be the label
# plt.text(0, percentiles_contention[0], str(np.round(percentiles_contention[0],2)), \
#          fontsize=12, verticalalignment='bottom')
# plt.tight_layout()


# plt.savefig(os.path.join(results_directory_experiment, percentile_filename))
# plt.show()


# slope = np.diff(percentiles_contention)/(np.diff(lambda_range)*(schedule_contention.end_time - schedule_contention.start_time))
# plt.title("Percentile latency slope")
# plt.xlabel("lambda*schedule_duration (us)")
# plt.ylabel(str(percentile_to_plot) + "percentile latency slope (us)")
# if scale == "log":
#         plt.yscale('log')
#         plt.ylim(10**-2, 10**2)
# plt.plot(np.array(lambda_range[1:])*(schedule_contention.end_time - schedule_contention.start_time), slope, ".-")
# plt.savefig(os.path.join(results_directory_experiment, percentile_slope_filename))

# print(slope)
# # Plot the mean latency curve

# plt.figure(figsize=(10, 8))
# # mean_latencies = []
# # for lambda_value in lambda_range:
# #     mean_latencies.append(results_allUEs_per_lambda_reserved[lambda_value]["mean_latency"])
# # plt.plot(np.array(lambda_range)*(schedule_reserved.end_time - schedule_reserved.start_time),\
# #          mean_latencies, ".-", label = "reserved")



# mean_latencies_contention = []
# mean_latencies_contention_std = []
# for lambda_value in lambda_range:
#     mean_latencies_contention.append(results_allUEs_per_lambda_contention[lambda_value]["mean_latency"])
#     mean_latencies_contention_std.append(\
#         results_allUEs_per_lambda_contention[lambda_value]["mean_latency_std"])
# plt.errorbar(np.array(lambda_range)*scaling_factor,\
#         mean_latencies_contention, mean_latencies_contention_std, label = "contention", fmt='.-', \
#         capsize=3)


# plt.text(0, mean_latencies_contention[0], str(np.round(mean_latencies_contention[0],2)), \
#          fontsize=12, verticalalignment='bottom')

# plt.legend()

# plt.xlabel("lambda (packets/s)")
# plt.ylabel("Mean latency (us)")

# if scale == "log":
#         plt.yscale('log')

# title = (f"Simulation 3 mean latency vs lambda, \n PER = {PER}, \n"
#          f"num_UEs: {num_UEs}, \n"
#          f"allowed_payload: {payload_size} B, \n "
#          f"packet size: {packet_sizes[0]} B, \n"
#          f"delivery_latency: {delivery_latency} us ,\n"
#          )
# plt.title(title)
# plt.tight_layout()

# plt.savefig(os.path.join(results_directory_experiment, mean_filename))


# plt.show()


# slope = np.diff(mean_latencies_contention)/(np.diff(lambda_range)*(schedule_contention.end_time - schedule_contention.start_time))
# plt.title("Mean latency slope")
# plt.xlabel("lambda*schedule_duration (us)")
# plt.ylabel("Mean percentile latency slope (us)")
# if scale == "log":
#         plt.yscale('log')
# plt.plot(np.array(lambda_range[1:])*(schedule_contention.end_time - schedule_contention.start_time), slope, ".-")
# plt.savefig(os.path.join(results_directory_experiment, mean_slope_filename))

# plt.figure(figsize=(10, 8))
# # mean_latencies = []
# # for lambda_value in lambda_range:
# #     mean_latencies.append(results_allUEs_per_lambda_reserved[lambda_value]["mean_latency"])
# # plt.plot(np.array(lambda_range)*(schedule_reserved.end_time - schedule_reserved.start_time),\
# #          mean_latencies, ".-", label = "reserved")

# # scale = "linear"


# unserved_packets_contention = []
# unserved_packets_contention_std = []
# for lambda_value in lambda_range:
#     unserved_packets_contention.append(results_allUEs_per_lambda_contention[lambda_value]["n_packets_not_served"])
#     unserved_packets_contention_std.append(\
#         results_allUEs_per_lambda_contention[lambda_value]["n_packets_not_served_std"])
# plt.errorbar(np.array(lambda_range)*scaling_factor,\
#         np.array(unserved_packets_contention) + 1, np.array(unserved_packets_contention_std) + 0.001, label = "contention", fmt='.-', \
#         capsize=3)


# # plt.text(0, unserved_packets_contention[0], str(np.round(unserved_packets_contention[0],2)), \
#         #  fontsize=12, verticalalignment='bottom')

# plt.legend()

# plt.xlabel("lambda (packets/s)")
# plt.ylabel("Unserved packets")

# if scale == "log":
#         plt.yscale('log')

# title = (f"Simulation 3 unserved vs lambda, \n PER = {PER}, \n"
#          f"num_UEs: {num_UEs}, \n"
#          f"allowed_payload: {payload_size} B, \n "
#          f"packet size: {packet_sizes[0]} B, \n"
#          f"delivery_latency: {delivery_latency} us ,\n"
#         )
# plt.title(title)
# plt.tight_layout()

# plt.savefig(os.path.join(results_directory_experiment, n_packets_not_served_filename))

# plt.show()




# bus_occupancy_contention = []
# for lambda_value in lambda_range:
#     bus_occupancy_contention.append(results_allUEs_per_lambda_contention[lambda_value]["bus_occupancy"])
# plt.plot(np.array(lambda_range)*scaling_factor, \
#         bus_occupancy_contention, '.-', label = "contention")
# # plt.plot(n_packets_generated, percentiles)
# plt.xlabel("lambda (packets/s)")
# plt.ylabel("Bus occupancy")
# plt.legend()

# if scale == "log":
#         plt.yscale('log')

# title = (f"Simulation 3 Bus occupancy vs lambda, \n PER = {PER},\n"
#          f"num_UEs: {num_UEs}, \n"
#          f"allowed_payload: {payload_size} B, \n "
#          f"packet size: {packet_sizes[0]} B, \n"
#          f"delivery_latency: {delivery_latency} us ,\n"
#         )
# plt.title(title)
# # Insert a textbox at the lowest y value of the plot and have y axis be the label
# plt.tight_layout()
# plt.savefig(os.path.join(results_directory_experiment, bus_occupancy_filename))
# plt.show()


# wins_contention = []
# for lambda_value in lambda_range:
#     wins_contention.append(results_allUEs_per_lambda_contention[lambda_value]["contention_wins"])
# plt.plot(np.array(lambda_range)*scaling_factor, \
#         wins_contention, '.-', label = "contention")
# # plt.plot(n_packets_generated, percentiles)
# plt.xlabel("lambda (packets/s)")
# plt.ylabel("Contention wins")
# plt.legend()

# if scale == "log":
#         plt.yscale('log')

# title = (f"Simulation 3 Bus occupancy vs lambda, \n PER = {PER},\n"
#          f"num_UEs: {num_UEs}, \n"
#          f"allowed_payload: {payload_size} B, \n "
#          f"packet size: {packet_sizes[0]} B, \n"
#          f"delivery_latency: {delivery_latency} us ,\n"
#         )
# plt.title(title)
# # Insert a textbox at the lowest y value of the plot and have y axis be the label
# plt.tight_layout()
# plt.savefig(os.path.join(results_directory_experiment, n_wins_filename))
# plt.show()